## This Jupyter is to test the Rating class


In [1]:
from rating import *

rating = Rating('2019-09-08 08:00:00', '2023-02-08 18:45:00')
df = rating.rate_stations()


In [2]:
df

,station_name,snr,std,snr_rating,std_rating
16,AUSTRIA-OE3FLB_55,1.19,0.41,4.14,1.66
87,MRT3_59,1.20,0.36,4.14,1.55
85,MRT3_02,1.19,0.38,4.13,1.60
86,MRT3_03,1.18,0.39,4.11,1.61
122,SWISS-BLEN7M_03,1.17,0.38,4.11,1.58
...,...,...,...,...,...
91,NORWAY-NY-AALESUND_01,0.48,0.77,1.78,2.01
109,SPAIN-SIGUENZA_01,0.44,2.54,1.68,4.03
67,KRIM_01,0.41,2.24,1.64,4.03
141,UNAM_59,0.40,2.38,1.63,4.00


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

date_today = datetime.now()
days = pd.date_range(date_today, date_today + timedelta(7), freq='D')

np.random.seed(seed=1111)
data = np.random.randint(1, high=100, size=len(days))
df = pd.DataFrame({'test': days, 'col2': data, 'col3': 2*data, 'col4': data**2})
df = df.set_index('test')
print(df)

                            col2  col3  col4
test                                        
2023-10-03 16:02:32.025711    29    58   841
2023-10-04 16:02:32.025711    56   112  3136
2023-10-05 16:02:32.025711    82   164  6724
2023-10-06 16:02:32.025711    13    26   169
2023-10-07 16:02:32.025711    35    70  1225
2023-10-08 16:02:32.025711    53   106  2809
2023-10-09 16:02:32.025711    25    50   625
2023-10-10 16:02:32.025711    23    46   529


In [11]:
(df.mean() / df.std()).mean()

1.470421312215637

In [12]:
df.mean() / df.std()

col2    1.747621
col3    1.747621
col4    0.916021
dtype: float64

In [15]:
(df.rolling(window=3).mean() / df.rolling(window=3).std()).mean(axis=1)

test
2023-10-03 16:02:32.025711         NaN
2023-10-04 16:02:32.025711         NaN
2023-10-05 16:02:32.025711    1.801336
2023-10-06 16:02:32.025711    1.302419
2023-10-07 16:02:32.025711    1.075903
2023-10-08 16:02:32.025711    1.471809
2023-10-09 16:02:32.025711    2.228520
2023-10-10 16:02:32.025711    1.679595
dtype: float64

In [4]:
import sys
import os
import astropy.io.fits
import numpy as np
import psycopg2.extras
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
import re
import datetime
import math
import mpu
import config as ecallisto_config

from collections import Counter
from daily_observations import DailyObservation
from astral import Observer
from astral.sun import sun

# sys.path.append(os.path.join(os.path.dirname(__file__), "../..", "radiospectra2"))
from radiospectra.sources import CallistoSpectrogram


class Rating:
    def __init__(self, start_time, end_time):
        self.start_time = start_time
        self.end_time = end_time
        self.database = self.get_db()

    def get_db(self):
        """Connect to the database"""
        return psycopg2.connect(host=ecallisto_config.DB_HOST,
                                user=ecallisto_config.DB_USER,
                                database=ecallisto_config.DB_DATABASE,
                                port=ecallisto_config.DB_PORT,
                                password=ecallisto_config.DB_PASSWORD)

    def get_all_instruments(self):
        """Get all data from the database"""
        sql_query_instruments = f"""
            SELECT path, snr, std 
            FROM test 
            WHERE start_time BETWEEN '{self.start_time}' AND '{self.end_time}' 
                AND snr IS NOT NULL 
            ORDER BY snr DESC;
        """
        with self.database.cursor(cursor_factory=psycopg2.extras.DictCursor) as cursor:
            cursor.execute(sql_query_instruments)
            instruments = cursor.fetchall()
        return instruments

    def convert_to_stars(self, score):
        ranking = [1.0, 2.0, 3.0, 4.0, 5.0]
        percent = np.percentile(score, [0, 25, 50, 75, 100])
        rating = np.interp(score, percent, ranking)
        return np.round(rating, 2)

    def rate_stations(self):
        instruments = self.get_all_instruments()
        df = pd.DataFrame(instruments, columns=["path", "snr", "std"])
        df = df[df['snr'].notna()]
        df["station_name"] = df["path"].str.split('[/|_]', expand=True)[4]
        df["snr_rating"] = self.convert_to_stars(df["snr"])
        df["std_rating"] = self.convert_to_stars(df["std"])
        df["counter"] = df["station_name"].tolist()
        values, counts = np.unique(df["counter"], return_counts=True)
        df = df.groupby('station_name').mean().round(decimals=2).sort_values(by='snr_rating', ascending=False)
        df["number_of_files"] = counts
        return df[["snr", "snr_rating", "std", "std_rating", "number_of_files"]]

    def combine_daily_observations(self):
        """
        Integrates DailyObservation with the rating system to provide a comprehensive analysis.
        """
        daily_observation = DailyObservation(self.start_time, self.end_time, "")
        daily_data = daily_observation.data

        ratings = self.rate_stations()

        # Merge daily observations with ratings
        combined_data = pd.merge(daily_data, ratings, left_on='Station', right_index=True, how='inner')
        return combined_data

    def calculate_total_duration(self):
        """
        Calculates the total duration and average SNR and std for each station.

        Returns:
            pandas.DataFrame: Total duration and average SNR and std data by station.
        """
        combined_data = self.combine_daily_observations()

        combined_data['Duration'] = pd.to_timedelta(combined_data['Duration'])
        total_duration = combined_data.groupby('Station')['Duration'].sum().reset_index()
        total_duration['Duration'] = total_duration['Duration'].apply(
            lambda duration: f"{duration.days} days, {str(duration)[-8:]}" if duration.days > 0 else str(duration)
        )

        average_stats = combined_data.groupby('Station').agg({'Avg-SNR': 'mean', 'Avg-STD': 'mean'}).reset_index()
        total_duration = total_duration.merge(average_stats, on='Station')
        return total_duration



rating = Rating('2021-09-08 08:00:00', '2021-09-08 18:45:00')
df_ratings = rating.rate_stations()
print(df_ratings)

                       snr  snr_rating   std  std_rating  number_of_files
station_name                                                             
MRT3                  1.21        4.78  0.37        1.41                8
ALASKA-COHOE          1.14        4.58  0.41        1.52               38
INDIA-OOTY            1.13        4.53  0.37        1.41               39
MRT2                  1.05        4.29  0.38        1.43               22
SWISS-IRSOL           1.04        4.20  0.44        1.59               30
MRT1                  0.99        4.11  1.47        4.08               37
SWISS-HB9SCT          1.01        4.07  0.71        2.37               73
GLASGOW               0.99        4.07  0.49        1.71               38
DENMARK               0.97        4.03  0.58        1.94                6
KRIM                  0.99        4.01  0.58        1.96               44
TRIEST                0.99        3.98  0.61        2.11               43
POLAND-Grotniki       0.93        3.84

/tmp/ipykernel_245430/1802454132.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('station_name').mean().round(decimals=2).sort_values(by='snr_rating', ascending=False)


In [5]:
combined_data = rating.combine_daily_observations()
print(combined_data)

Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calc

/tmp/ipykernel_245430/1802454132.py:67: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby('station_name').mean().round(decimals=2).sort_values(by='snr_rating', ascending=False)


In [6]:
total_duration_data = rating.calculate_total_duration()
print(total_duration_data)

Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calculating sunrise/sunset: Sun never reaches 6 degrees below the horizon, at this location.
Error calc

KeyboardInterrupt: 